In [ ]:
import uproot
import awkward as ak
import numpy as np
import sklearn.metrics as m
import boost_histogram as bh
import glob
import os

from matplotlib import pyplot as plt
import matplotlib as mpl
from cycler import cycler
import mplhep as hep

use_helvet = False  ## true: use helvetica for plots, make sure the system have the font installed
if use_helvet:
    CMShelvet = hep.style.CMS
    CMShelvet["font.sans-serif"] = ["Helvetica", "Arial"]
    plt.style.use(CMShelvet)
else:
    plt.style.use(hep.style.CMS)

# ROC

In [ ]:
def getConfig(opt, pt="low", mass="higgs", year="2017"):
    from types import SimpleNamespace

    if pt == "lower":
        ptmin, ptmax = 200, 400
    elif pt == "low":
        ptmin, ptmax = 400, 600
    elif pt == "high":
        ptmin, ptmax = 600, 1000
    elif pt == "higher":
        ptmin, ptmax = 1000, 100000
    elif pt == "full":
        ptmin, ptmax = 200, 100000

    if mass == "higgs":
        massmin, massmax = 60, 150
    if mass == "wz":
        massmin, massmax = 50, 140
    elif mass == "x50":
        massmin, massmax = 0, 80
    elif mass == "x250":
        massmin, massmax = 150, 300
    elif mass == "x350":
        massmin, massmax = 250, 400
    elif mass == "full":
        massmin, massmax = 50, 250
    elif mass == "fullmax":
        massmin, massmax = 50, 800
    elif isinstance(mass, tuple):
        massmin, massmax = mass

    routine, sname, bname = opt.split(":")
    routine_config = None
    if ";" in routine:
        routine, routine_config = routine.split(";")
    cfg = SimpleNamespace(
        year=year,
        ptmin=ptmin,
        ptmax=ptmax,
        filedir=None,
        filelist={
            "sig": None,
            "bkg": None,
        },
        base_cut=f"(fj_pt>{ptmin}) & (fj_pt<{ptmax}) & (abs(fj_eta)<2.4)",
        mass_cut=f"(fj_sdmass>{massmin}) & (fj_sdmass<{massmax})",
        subtitle=(
            (r"$%d < p_{T} < %d$ GeV,  $|\eta|<2.4$" % (ptmin, ptmax))
            if ptmax != 100000
            else r"$p_{T,j} > %d$ GeV,  $|\eta_{j}|<2.4$" % (ptmin)
        ),
        subtitle2=r"$%d < m_{SD} < %d$ GeV" % (massmin, massmax),
        label_list=[
            "label_Top_bWcs",
            "label_Top_bWqq",
            "label_Top_bWc",
            "label_Top_bWs",
            "label_Top_bWq",
            "label_Top_bWev",
            "label_Top_bWmv",
            "label_Top_bWtauev",
            "label_Top_bWtaumv",
            "label_Top_bWtauhv",
            "label_Top_Wcs",
            "label_Top_Wqq",
            "label_Top_Wev",
            "label_Top_Wmv",
            "label_Top_Wtauev",
            "label_Top_Wtaumv",
            "label_Top_Wtauhv",
            "label_H_bb",
            "label_H_cc",
            "label_H_ss",
            "label_H_qq",
            "label_H_bc",
            "label_H_bs",
            "label_H_cs",
            "label_H_gg",
            "label_H_ee",
            "label_H_mm",
            "label_H_tauhtaue",
            "label_H_tauhtaum",
            "label_H_tauhtauh",
            "label_H_WW_cscs",
            "label_H_WW_csqq",
            "label_H_WW_qqqq",
            "label_H_WW_csc",
            "label_H_WW_css",
            "label_H_WW_csq",
            "label_H_WW_qqc",
            "label_H_WW_qqs",
            "label_H_WW_qqq",
            "label_H_WW_csev",
            "label_H_WW_qqev",
            "label_H_WW_csmv",
            "label_H_WW_qqmv",
            "label_H_WW_cstauev",
            "label_H_WW_qqtauev",
            "label_H_WW_cstaumv",
            "label_H_WW_qqtaumv",
            "label_H_WW_cstauhv",
            "label_H_WW_qqtauhv",
            "label_H_WxWx_cscs",
            "label_H_WxWx_csqq",
            "label_H_WxWx_qqqq",
            "label_H_WxWx_csc",
            "label_H_WxWx_css",
            "label_H_WxWx_csq",
            "label_H_WxWx_qqc",
            "label_H_WxWx_qqs",
            "label_H_WxWx_qqq",
            "label_H_WxWx_csev",
            "label_H_WxWx_qqev",
            "label_H_WxWx_csmv",
            "label_H_WxWx_qqmv",
            "label_H_WxWx_cstauev",
            "label_H_WxWx_qqtauev",
            "label_H_WxWx_cstaumv",
            "label_H_WxWx_qqtaumv",
            "label_H_WxWx_cstauhv",
            "label_H_WxWx_qqtauhv",
            "label_H_WxWxStar_cscs",
            "label_H_WxWxStar_csqq",
            "label_H_WxWxStar_qqqq",
            "label_H_WxWxStar_csc",
            "label_H_WxWxStar_css",
            "label_H_WxWxStar_csq",
            "label_H_WxWxStar_qqc",
            "label_H_WxWxStar_qqs",
            "label_H_WxWxStar_qqq",
            "label_H_WxWxStar_csev",
            "label_H_WxWxStar_qqev",
            "label_H_WxWxStar_csmv",
            "label_H_WxWxStar_qqmv",
            "label_H_WxWxStar_cstauev",
            "label_H_WxWxStar_qqtauev",
            "label_H_WxWxStar_cstaumv",
            "label_H_WxWxStar_qqtaumv",
            "label_H_WxWxStar_cstauhv",
            "label_H_WxWxStar_qqtauhv",
            "label_H_ZZ_bbbb",
            "label_H_ZZ_bbcc",
            "label_H_ZZ_bbss",
            "label_H_ZZ_bbqq",
            "label_H_ZZ_cccc",
            "label_H_ZZ_ccss",
            "label_H_ZZ_ccqq",
            "label_H_ZZ_ssss",
            "label_H_ZZ_ssqq",
            "label_H_ZZ_qqqq",
            "label_H_ZZ_bbb",
            "label_H_ZZ_bbc",
            "label_H_ZZ_bbs",
            "label_H_ZZ_bbq",
            "label_H_ZZ_ccb",
            "label_H_ZZ_ccc",
            "label_H_ZZ_ccs",
            "label_H_ZZ_ccq",
            "label_H_ZZ_ssb",
            "label_H_ZZ_ssc",
            "label_H_ZZ_sss",
            "label_H_ZZ_ssq",
            "label_H_ZZ_qqb",
            "label_H_ZZ_qqc",
            "label_H_ZZ_qqs",
            "label_H_ZZ_qqq",
            "label_H_ZZ_bbee",
            "label_H_ZZ_bbmm",
            "label_H_ZZ_bbe",
            "label_H_ZZ_bbm",
            "label_H_ZZ_bee",
            "label_H_ZZ_bmm",
            "label_H_ZZ_bbtauhtaue",
            "label_H_ZZ_bbtauhtaum",
            "label_H_ZZ_bbtauhtauh",
            "label_H_ZZ_btauhtaue",
            "label_H_ZZ_btauhtaum",
            "label_H_ZZ_btauhtauh",
            "label_H_ZZ_ccee",
            "label_H_ZZ_ccmm",
            "label_H_ZZ_cce",
            "label_H_ZZ_ccm",
            "label_H_ZZ_cee",
            "label_H_ZZ_cmm",
            "label_H_ZZ_cctauhtaue",
            "label_H_ZZ_cctauhtaum",
            "label_H_ZZ_cctauhtauh",
            "label_H_ZZ_ctauhtaue",
            "label_H_ZZ_ctauhtaum",
            "label_H_ZZ_ctauhtauh",
            "label_H_ZZ_ssee",
            "label_H_ZZ_ssmm",
            "label_H_ZZ_sse",
            "label_H_ZZ_ssm",
            "label_H_ZZ_see",
            "label_H_ZZ_smm",
            "label_H_ZZ_sstauhtaue",
            "label_H_ZZ_sstauhtaum",
            "label_H_ZZ_sstauhtauh",
            "label_H_ZZ_stauhtaue",
            "label_H_ZZ_stauhtaum",
            "label_H_ZZ_stauhtauh",
            "label_H_ZZ_qqee",
            "label_H_ZZ_qqmm",
            "label_H_ZZ_qqe",
            "label_H_ZZ_qqm",
            "label_H_ZZ_qee",
            "label_H_ZZ_qmm",
            "label_H_ZZ_qqtauhtaue",
            "label_H_ZZ_qqtauhtaum",
            "label_H_ZZ_qqtauhtauh",
            "label_H_ZZ_qtauhtaue",
            "label_H_ZZ_qtauhtaum",
            "label_H_ZZ_qtauhtauh",
            "label_H_ZxZx_bbbb",
            "label_H_ZxZx_bbcc",
            "label_H_ZxZx_bbss",
            "label_H_ZxZx_bbqq",
            "label_H_ZxZx_cccc",
            "label_H_ZxZx_ccss",
            "label_H_ZxZx_ccqq",
            "label_H_ZxZx_ssss",
            "label_H_ZxZx_ssqq",
            "label_H_ZxZx_qqqq",
            "label_H_ZxZx_bbb",
            "label_H_ZxZx_bbc",
            "label_H_ZxZx_bbs",
            "label_H_ZxZx_bbq",
            "label_H_ZxZx_ccb",
            "label_H_ZxZx_ccc",
            "label_H_ZxZx_ccs",
            "label_H_ZxZx_ccq",
            "label_H_ZxZx_ssb",
            "label_H_ZxZx_ssc",
            "label_H_ZxZx_sss",
            "label_H_ZxZx_ssq",
            "label_H_ZxZx_qqb",
            "label_H_ZxZx_qqc",
            "label_H_ZxZx_qqs",
            "label_H_ZxZx_qqq",
            "label_H_ZxZx_bbee",
            "label_H_ZxZx_bbmm",
            "label_H_ZxZx_bbe",
            "label_H_ZxZx_bbm",
            "label_H_ZxZx_bee",
            "label_H_ZxZx_bmm",
            "label_H_ZxZx_bbtauhtaue",
            "label_H_ZxZx_bbtauhtaum",
            "label_H_ZxZx_bbtauhtauh",
            "label_H_ZxZx_btauhtaue",
            "label_H_ZxZx_btauhtaum",
            "label_H_ZxZx_btauhtauh",
            "label_H_ZxZx_ccee",
            "label_H_ZxZx_ccmm",
            "label_H_ZxZx_cce",
            "label_H_ZxZx_ccm",
            "label_H_ZxZx_cee",
            "label_H_ZxZx_cmm",
            "label_H_ZxZx_cctauhtaue",
            "label_H_ZxZx_cctauhtaum",
            "label_H_ZxZx_cctauhtauh",
            "label_H_ZxZx_ctauhtaue",
            "label_H_ZxZx_ctauhtaum",
            "label_H_ZxZx_ctauhtauh",
            "label_H_ZxZx_ssee",
            "label_H_ZxZx_ssmm",
            "label_H_ZxZx_sse",
            "label_H_ZxZx_ssm",
            "label_H_ZxZx_see",
            "label_H_ZxZx_smm",
            "label_H_ZxZx_sstauhtaue",
            "label_H_ZxZx_sstauhtaum",
            "label_H_ZxZx_sstauhtauh",
            "label_H_ZxZx_stauhtaue",
            "label_H_ZxZx_stauhtaum",
            "label_H_ZxZx_stauhtauh",
            "label_H_ZxZx_qqee",
            "label_H_ZxZx_qqmm",
            "label_H_ZxZx_qqe",
            "label_H_ZxZx_qqm",
            "label_H_ZxZx_qee",
            "label_H_ZxZx_qmm",
            "label_H_ZxZx_qqtauhtaue",
            "label_H_ZxZx_qqtauhtaum",
            "label_H_ZxZx_qqtauhtauh",
            "label_H_ZxZx_qtauhtaue",
            "label_H_ZxZx_qtauhtaum",
            "label_H_ZxZx_qtauhtauh",
            "label_H_ZxZxStar_bbbb",
            "label_H_ZxZxStar_bbcc",
            "label_H_ZxZxStar_bbss",
            "label_H_ZxZxStar_bbqq",
            "label_H_ZxZxStar_cccc",
            "label_H_ZxZxStar_ccss",
            "label_H_ZxZxStar_ccqq",
            "label_H_ZxZxStar_ssss",
            "label_H_ZxZxStar_ssqq",
            "label_H_ZxZxStar_qqqq",
            "label_H_ZxZxStar_bbb",
            "label_H_ZxZxStar_bbc",
            "label_H_ZxZxStar_bbs",
            "label_H_ZxZxStar_bbq",
            "label_H_ZxZxStar_ccb",
            "label_H_ZxZxStar_ccc",
            "label_H_ZxZxStar_ccs",
            "label_H_ZxZxStar_ccq",
            "label_H_ZxZxStar_ssb",
            "label_H_ZxZxStar_ssc",
            "label_H_ZxZxStar_sss",
            "label_H_ZxZxStar_ssq",
            "label_H_ZxZxStar_qqb",
            "label_H_ZxZxStar_qqc",
            "label_H_ZxZxStar_qqs",
            "label_H_ZxZxStar_qqq",
            "label_H_ZxZxStar_bbee",
            "label_H_ZxZxStar_bbmm",
            "label_H_ZxZxStar_bbe",
            "label_H_ZxZxStar_bbm",
            "label_H_ZxZxStar_bee",
            "label_H_ZxZxStar_bmm",
            "label_H_ZxZxStar_bbtauhtaue",
            "label_H_ZxZxStar_bbtauhtaum",
            "label_H_ZxZxStar_bbtauhtauh",
            "label_H_ZxZxStar_btauhtaue",
            "label_H_ZxZxStar_btauhtaum",
            "label_H_ZxZxStar_btauhtauh",
            "label_H_ZxZxStar_ccee",
            "label_H_ZxZxStar_ccmm",
            "label_H_ZxZxStar_cce",
            "label_H_ZxZxStar_ccm",
            "label_H_ZxZxStar_cee",
            "label_H_ZxZxStar_cmm",
            "label_H_ZxZxStar_cctauhtaue",
            "label_H_ZxZxStar_cctauhtaum",
            "label_H_ZxZxStar_cctauhtauh",
            "label_H_ZxZxStar_ctauhtaue",
            "label_H_ZxZxStar_ctauhtaum",
            "label_H_ZxZxStar_ctauhtauh",
            "label_H_ZxZxStar_ssee",
            "label_H_ZxZxStar_ssmm",
            "label_H_ZxZxStar_sse",
            "label_H_ZxZxStar_ssm",
            "label_H_ZxZxStar_see",
            "label_H_ZxZxStar_smm",
            "label_H_ZxZxStar_sstauhtaue",
            "label_H_ZxZxStar_sstauhtaum",
            "label_H_ZxZxStar_sstauhtauh",
            "label_H_ZxZxStar_stauhtaue",
            "label_H_ZxZxStar_stauhtaum",
            "label_H_ZxZxStar_stauhtauh",
            "label_H_ZxZxStar_qqee",
            "label_H_ZxZxStar_qqmm",
            "label_H_ZxZxStar_qqe",
            "label_H_ZxZxStar_qqm",
            "label_H_ZxZxStar_qee",
            "label_H_ZxZxStar_qmm",
            "label_H_ZxZxStar_qqtauhtaue",
            "label_H_ZxZxStar_qqtauhtaum",
            "label_H_ZxZxStar_qqtauhtauh",
            "label_H_ZxZxStar_qtauhtaue",
            "label_H_ZxZxStar_qtauhtaum",
            "label_H_ZxZxStar_qtauhtauh",
            "label_QCD_bb",
            "label_QCD_cc",
            "label_QCD_b",
            "label_QCD_c",
            "label_QCD_others",
            "btaue",
            "btaum",
            "btauh",
            "aa",
            "x5",
            "x6",
            "x7",
            "x8",
            "x9",
            "x10",
            "x11",
            "x12",
            "x13",
            "x14",
            "x15",
            "x16",
            "x17",
            "x18",
            "x19",
            "x20",
            "x21",
            "x22",
            "x23",
            "x24",
            "x25",
            "x26",
            "x27",
            "x28",
            "x29",
            "x30",
            "aabb",
            "aacc",
            "aass",
            "aaqq",
            "aabc",
            "aacs",
            "aabq",
            "aacq",
            "aasq",
            "aagg",
            "aaee",
            "aamm",
            "aatauhtaue",
            "aatauhtaum",
            "aatauhtauh",
            "aab",
            "aac",
            "aas",
            "aaq",
            "aag",
            "aae",
            "aam",
            "aataue",
            "aataum",
            "aatauh",
            "abb",
            "acc",
            "ass",
            "aqq",
            "abc",
            "acs",
            "abq",
            "acq",
            "asq",
            "agg",
            "aee",
            "amm",
            "atauhtaue",
            "atauhtaum",
            "atauhtauh",
        ],  # this is the idx-name mapping from dnntuples, not the actual label for tagger training
    )
    cfg.label_idx = {lab: i for i, lab in enumerate(cfg.label_list)}

    if routine == "v3":
        # cfg.label = 'GloParT 3'
        cfg.label = "GloParT"
        cfg.filedir = "/ceph/cms/store/user/woodson/glopart-plotter/predict/ak8_MD_inclv10beta4_ul_manual.nlayer10.vispart_as_resid.ddp4-bs640-lr1p2e-3.nepoch100.farm221.best_epoch"

    # previous versions
    elif routine == "v2":
        cfg.label = f"GloParT 2"
        cfg.filedir = "/ceph/cms/store/user/woodson/glopart-plotter/predict/ak8_MD_inclv8_part_addltphp_wmeasonly_manual.useamp.large.gm5.ddp-bs256-lr2e-3/new"

    elif routine == "PNetMD":
        # cfg.label = 'ParticleNet-MD (legacy)'
        cfg.label = "ParticleNet"
        cfg.filedir = "/ceph/cms/store/user/woodson/glopart-plotter/predict/ak8_MD_inclv10beta4_ul_manual.nlayer10.vispart_as_resid.ddp4-bs640-lr1p2e-3.nepoch100.farm221.best_epoch"

    else:
        raise RuntimeError("wrong opt")

    # determine which file to use
    year_file_id = {
        "2017": "",
        "2023": "run3_2023_",
        "2023BPix": "run3_2023bpix_",
        "2023FixLTS": "run3_2023_fixlts_",
        "2017PUPPIv18": "run2_repuppi_",
    }[year]
    file_regex_map = {
        "QCD": f"pred_{year_file_id}qcd[0-9]*.root",  # always read split QCD files
        "QCDLO": f"pred_{year_file_id}qcd470to600.root",  # always read split QCD files
        "QCDHI": f"pred_{year_file_id}qcd1000to1400.root",  # always read split QCD files
        "HIGLO": f"pred_{year_file_id}higlo*.root",  # possible to have split files (part1, part2, ...)
        "HIGHI": f"pred_{year_file_id}highi*.root",
        "HWWLO": f"pred_{year_file_id}hwwlo*.root",
        "HWWHI": f"pred_{year_file_id}hwwhi*.root",
        "SMTTBARSL": f"pred_{year_file_id}ofcttbarsl.root",
        **{
            n: f"pred_{year_file_id}{n.lower()}.root"
            for n in ["HIGGS2P", "HWW", "TTBARINCL", "TTBAR", "WLO", "WHI", "ZLO", "ZHI"]
        },
    }
    name_choices = [
        "HIGLO",
        "HIGHI",
        "HWWLO",
        "HWWHI",
        "HIGGS2P",
        "HWW",
        "QCDLO",
        "QCDHI",
        "QCD",
        "SMTTBARSL",
        "TTBARINCL",
        "TTBAR",
        "WLO",
        "WHI",
        "ZLO",
        "ZHI",
    ]
    cfg.filelist["sig"] = glob.glob(
        [os.path.join(cfg.filedir, file_regex_map[n]) for n in name_choices if sname.startswith(n)][
            0
        ]
    )
    cfg.filelist["bkg"] = glob.glob(
        [os.path.join(cfg.filedir, file_regex_map[n]) for n in name_choices if bname.startswith(n)][
            0
        ]
    )
    sname = (
        sname.replace("QCDLO", "QCD")
        .replace("QCDHI", "QCD")
        .replace("HIGLO", "HIG")
        .replace("HIGHI", "HIG")
        .replace("HWWLO", "HWW")
        .replace("HWWHI", "HWW")
        .replace("WLO", "W")
        .replace("WHI", "W")
        .replace("ZLO", "Z")
        .replace("ZHI", "Z")
    )
    bname = (
        bname.replace("QCDLO", "QCD")
        .replace("QCDHI", "QCD")
        .replace("HIGLO", "HIG")
        .replace("HIGHI", "HIG")
        .replace("HWWLO", "HWW")
        .replace("HWWHI", "HWW")
        .replace("WLO", "W")
        .replace("WHI", "W")
        .replace("ZLO", "Z")
        .replace("ZHI", "Z")
    )

    # deal with variables
    variable_dict = {
        "QCD": ["label_QCD_bb", "label_QCD_cc", "label_QCD_b", "label_QCD_c", "label_QCD_others"],
        **{
            f"HIG{n}": [f"label_H_{n}"]
            for n in [
                "bb",
                "cc",
                "qq",
                "ss",
                "bc",
                "cs",
                "bs",
                "gg",
                "ee",
                "mm",
                "tauhtaue",
                "tauhtaum",
                "tauhtauh",
            ]
        },
        **{
            f"HIGGS2P{n}": [f"label_H_{n}"]
            for n in [
                "bb",
                "cc",
                "qq",
                "ss",
                "bc",
                "cs",
                "bs",
                "gg",
                "ee",
                "mm",
                "tauhtaue",
                "tauhtaum",
                "tauhtauh",
            ]
        },
        "HIGll": [f"label_H_{n}" for n in ["ss", "qq"]],
        **{
            f"HWW{n}": [f"label_H_WW_{n}"]
            for n in [
                "cscs",
                "csqq",
                "qqqq",
                "csc",
                "css",
                "csq",
                "qqc",
                "qqs",
                "qqq",
                "csev",
                "qqev",
                "csmv",
                "qqmv",
                "cstauev",
                "qqtauev",
                "cstaumv",
                "qqtaumv",
                "cstauhv",
                "qqtauhv",
            ]
        },
        **{
            f"TTBAR{n}": [f"label_Top_{n}"]
            for n in [
                "bWcs",
                "bWqq",
                "bWc",
                "bWs",
                "bWq",
                "bWev",
                "bWmv",
                "bWtauev",
                "bWtaumv",
                "bWtauhv",
                "Wcs",
                "Wqq",
                "Wev",
                "Wmv",
                "Wtauev",
                "Wtaumv",
                "Wtauhv",
            ]
        },
        **{f"W{n}": [f"label_H_{n}"] for n in ["qq", "cs"]},
        **{f"Z{n}": [f"label_H_{n}"] for n in ["bb", "cc", "qq", "ss"]},
        "HWWQQQQ": [f"label_H_WW_{n}" for n in ["cscs", "csqq", "qqqq"]],
        "HWWQQQ": [f"label_H_WW_{n}" for n in ["csc", "css", "csq", "qqc", "qqs", "qqq"]],
        "HWWQQev": [f"label_H_WW_{n}" for n in ["csev", "qqev"]],
        "HWWQQmv": [f"label_H_WW_{n}" for n in ["csmv", "qqmv"]],
        "HWWQQtauev": [f"label_H_WW_{n}" for n in ["cstauev", "qqtauev"]],
        "HWWQQtaumv": [f"label_H_WW_{n}" for n in ["cstaumv", "qqtaumv"]],
        "HWWQQtauhv": [f"label_H_WW_{n}" for n in ["cstauhv", "qqtauhv"]],
        "HWWQQta": [
            f"label_H_WW_{n}"
            for n in ["cstauev", "qqtauev", "cstaumv", "qqtaumv", "cstauhv", "qqtauhv"]
        ],
        # 'HWWel': [f'label_H_WW_{n}' for n in ['csev', 'cstauev', 'qqev', 'qqtauev']],
        # 'HWWmu': [f'label_H_WW_{n}' for n in ['csmv', 'cstaumv', 'qqmv', 'qqtaumv']],
        "TTBARbWall": [
            f"label_Top_{n}"
            for n in [
                "bWcs",
                "bWqq",
                "bWc",
                "bWs",
                "bWq",
                "bWev",
                "bWmv",
                "bWtauev",
                "bWtaumv",
                "bWtauhv",
            ]
        ],
        "TTBARbWhad": [f"label_Top_{n}" for n in ["bWcs", "bWqq", "bWc", "bWs", "bWq"]],
        "TTBARbWQQ": [f"label_Top_{n}" for n in ["bWcs", "bWqq"]],
        "TTBARbWta": [f"label_Top_{n}" for n in ["bWtauev", "bWtaumv", "bWtauhv"]],
        "TTBARbWtauhv": [f"label_Top_{n}" for n in ["bWtauhv"]],
        "TTBARall": [
            f"label_Top_{n}"
            for n in [
                "bWcs",
                "bWqq",
                "bWc",
                "bWs",
                "bWq",
                "bWev",
                "bWmv",
                "bWtauev",
                "bWtaumv",
                "bWtauhv",
                "Wcs",
                "Wqq",
                "Wev",
                "Wmv",
                "Wtauev",
                "Wtaumv",
                "Wtauhv",
            ]
        ],
        "WQQ": [f"label_H_{n}" for n in ["cs", "qq"]],
        "ZQQ": [f"label_H_{n}" for n in ["bb", "cc", "ss", "qq"]],
        "Zll": [f"label_H_{n}" for n in ["ss", "qq"]],
        "SMTTBARSLQQ": [f"label_Top_W{n}" for n in ["cs", "qq"]],
    }
    variable_dict.update(
        {
            k.replace("TTBAR", "SMTTBARSL", 1): v
            for k, v in variable_dict.items()
            if k.startswith("TTBAR")
        }
    )

    label_dict = {
        "QCD": "QCD",
        "HWWQQQQ": r"H$\rightarrow$WW 4q",
        "HWWQQQ": r"H$\rightarrow$WW 3q",
        "HWWQQev": r"H$\rightarrow$WW $e\nu qq$",
        "HWWQQmv": r"H$\rightarrow$WW $\mu\nu qq$",
        "HWWQQta": r"H$\rightarrow$WW $\tau\nu qq$",
        "HWWQQtauev": r"H$\rightarrow$WW $\tau_e\nu qq$",
        "HWWQQtaumv": r"H$\rightarrow$WW $\tau_\mu\nu qq$",
        "HWWQQtauhv": r"H$\rightarrow$WW $\tau_h\nu qq$",
        "HIGbb": r"H$\rightarrow$bb",
        "HIGcc": r"H$\rightarrow$cc",
        "HIGss": r"H$\rightarrow$ss",
        "HIGqq": r"H$\rightarrow$qq",
        "HIGbc": r"$H^\pm \rightarrow$bc",
        "HIGcs": r"$H^\pm \rightarrow$cs",
        "HIGbs": r"$H\rightarrow$bs",
        "HIGgg": r"$H\rightarrow$gg",
        "HIGee": r"$H\rightarrow$ee",
        "HIGmm": r"$H\rightarrow \mu\mu$",
        "HIGtauhvtauev": r"H$\rightarrow\tau_h\tau_e$",
        "HIGtauhvtaumv": r"H$\rightarrow\tau_h\tau_\mu$",
        "HIGtauhvtauhv": r"H$\rightarrow\tau_h\tau_h$",
        "HIGll": r"H$\rightarrow$qq (q=u/d/s)",
        "HIGtauhtaue": r"H$\rightarrow\tau_h\tau_e$",
        "HIGtauhtaum": r"H$\rightarrow\tau_h\tau_\mu$",
        "HIGtauhtauh": r"H$\rightarrow\tau_h\tau_h$",
        "HIGGS2Pbb": r"H$\rightarrow$bb",
        "HIGGS2Pcc": r"H$\rightarrow$cc",
        "HIGGS2Pss": r"H$\rightarrow$ss",
        "HIGGS2Pqq": r"H$\rightarrow$qq",
        "HIGGS2Pbc": r"$H^\pm \rightarrow$bc",
        "HIGGS2Pcs": r"$H^\pm \rightarrow$cs",
        "HIGGS2Pbs": r"$H\rightarrow$bs",
        "HIGGS2Pgg": r"$H\rightarrow$gg",
        "HIGGS2Pee": r"$H\rightarrow$ee",
        "HIGGS2Pmm": r"$H\rightarrow \mu\mu$",
        "HIGGS2Ptauhvtauev": r"H$\rightarrow\tau_h\tau_e$",
        "HIGGS2Ptauhvtaumv": r"H$\rightarrow\tau_h\tau_\mu$",
        "HIGGS2Ptauhvtauhv": r"H$\rightarrow\tau_h\tau_h$",
        "HIGGS2Ptauhtaue": r"H$\rightarrow\tau_h\tau_e$",
        "HIGGS2Ptauhtaum": r"H$\rightarrow\tau_h\tau_\mu$",
        "HIGGS2Ptauhtauh": r"H$\rightarrow\tau_h\tau_h$",
        "TTBARbWall": r"t$\rightarrow$bW all",
        "TTBARbWhad": r"t$\rightarrow$bW had.",
        "TTBARbWQQ": r"t$\rightarrow bW \rightarrow bqq$",
        "TTBARbWev": r"t$\rightarrow bW \rightarrow be\nu$",
        "TTBARbWmv": r"t$\rightarrow bW \rightarrow b\mu\nu$",
        "TTBARbWta": r"t$\rightarrow bW \rightarrow b\tau\nu$",
        "TTBARbWtauhv": r"t$\rightarrow bW \rightarrow b\tau_h\nu$",
        "TTBARINCLall": r"t$\rightarrow$bW all (incl.)",
        "TTBARINCLhad": r"t$\rightarrow$bW had. (incl.)",
        "TTBARINCLel": r"t$\rightarrow bW \rightarrow be\nu$ (incl.)",
        "TTBARINCLmu": r"t$\rightarrow bW \rightarrow b\mu\nu$ (incl.)",
        "SMTTBARSLbWall": r"t$\rightarrow$bW all",
        "SMTTBARSLbWhad": r"t$\rightarrow$bW had.",
        "SMTTBARSLbWQQ": r"t$\rightarrow bW \rightarrow bqq$",
        "SMTTBARSLbWev": r"t$\rightarrow bW \rightarrow be\nu$",
        "SMTTBARSLbWmv": r"t$\rightarrow bW \rightarrow b\mu\nu$",
        "SMTTBARSLbWta": r"t$\rightarrow bW \rightarrow b\tau\nu$",
        "SMTTBARSLbWcs": r"t$\rightarrow bW \rightarrow bcs$",
        "SMTTBARSLbWqq": r"t$\rightarrow bW \rightarrow bqq$",
        "SMTTBARSLQQ": r"W$\rightarrow$qq (from $t\overline{t}$)",
        "XBCbc": r"$H\prime\rightarrow$bc",
        "Wcs": r"W$\rightarrow$cs",
        "Wqq": r"W$\rightarrow$qq",
        "Zbb": r"Z$\rightarrow$bb",
        "Zcc": r"Z$\rightarrow$cc",
        "Zqq": r"Z$\rightarrow$qq",
        "Zss": r"Z$\rightarrow$ss",
        "Zll": r"Z$\rightarrow$qq (q=u/d/s)",
        "WQQ": r"W$\rightarrow$qq (all)",
        "ZQQ": r"Z$\rightarrow$qq (all)",
    }
    cfg.y_sig_flag = " | ".join(
        [f"(fj_label == {cfg.label_idx[lab]})" for lab in variable_dict[sname]]
    )
    cfg.y_bkg_flag = " | ".join(
        [f"(fj_label == {cfg.label_idx[lab]})" for lab in variable_dict[bname]]
    )

    cfg.y_score_sig = " + ".join([f"score_{lab}" for lab in variable_dict[sname]])
    cfg.y_score_bkg = " + ".join([f"score_{lab}" for lab in variable_dict[bname]])
    cfg.y_score = f"({cfg.y_score_sig}) / (({cfg.y_score_sig}) + ({cfg.y_score_bkg}))"

    cfg.title = f"{label_dict[sname]} vs {label_dict[bname]}"

    # special treatment
    import re

    cfg.y_sig_flag = cfg.y_sig_flag + " "
    cfg.y_bkg_flag = cfg.y_bkg_flag + " "
    cfg.y_score = cfg.y_score + " "

    # speical treatment on variables
    if routine == "PNetMD":
        exist_variable_dict = {
            "QCD": " + ".join(
                [
                    f"pfMassDecorrelatedParticleNetJetTags_probQCD{n}"
                    for n in ["b", "bb", "c", "cc", "others"]
                ]
            ),
            **{
                f"HIGGS2P{n}": f"pfMassDecorrelatedParticleNetJetTags_probX{n}"
                for n in ["bb", "cc", "qq"]
            },
            **{
                f"HIG{n}": f"pfMassDecorrelatedParticleNetJetTags_probX{n}"
                for n in ["bb", "cc", "qq"]
            },
            **{
                f"Z{n}": f"pfMassDecorrelatedParticleNetJetTags_probX{n}"
                for n in ["bb", "cc", "qq"]
            },
            "ZQQ": " + ".join(
                [f"pfMassDecorrelatedParticleNetJetTags_probX{n}" for n in ["bb", "cc", "qq"]]
            ),
            "Zll": " + ".join([f"pfMassDecorrelatedParticleNetJetTags_probX{n}" for n in ["qq"]]),
            "WQQ": " + ".join(
                [f"pfMassDecorrelatedParticleNetJetTags_probX{n}" for n in ["cc", "qq"]]
            ),  # W->qq tagger: use cc+qq
            "Wcs": " + ".join(
                [f"pfMassDecorrelatedParticleNetJetTags_probX{n}" for n in ["cc"]]
            ),  # W->cs tagger: use cc
            "Wqq": " + ".join([f"pfMassDecorrelatedParticleNetJetTags_probX{n}" for n in ["qq"]]),
            "SMTTBARSLQQ": " + ".join(
                [f"pfMassDecorrelatedParticleNetJetTags_probX{n}" for n in ["cc", "qq"]]
            ),  # W->qq tagger: use cc+qq
        }
        cfg.y_score = f"({exist_variable_dict[sname]}) / (({exist_variable_dict[sname]}) + ({exist_variable_dict[bname]}))"
    if routine == "PNetRun3MD":
        exist_variable_dict = {
            ("HIGbb", "QCD"): "pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HbbvsQCD",
            ("HIGcc", "QCD"): "pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HccvsQCD",
            (
                "HIGcc",
                "HIGbb",
            ): "(pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HccvsQCD / (1 - pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HccvsQCD)) / (pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HccvsQCD / (1 - pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HccvsQCD) + pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HbbvsQCD / (1 - pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HbbvsQCD) + 1e-8)",
            ("HIGtauhtaue", "QCD"): "pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HtevsQCD",
            ("HIGtauhtaum", "QCD"): "pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HtmvsQCD",
            ("HIGtauhtauh", "QCD"): "pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HttvsQCD",
            (
                "HIGtauhtauh",
                "HIGll",
            ): "(pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HttvsQCD / (1 - pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HttvsQCD + 1e-8)) / (pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HttvsQCD / (1 - pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HttvsQCD + 1e-8) + pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HqqvsQCD / (1 - pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HqqvsQCD + 1e-8) + 1e-8)",
            (
                "HIGtauhtauh",
                "HIGbb",
            ): "(pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HttvsQCD / (1 - pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HttvsQCD + 1e-8)) / (pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HttvsQCD / (1 - pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HttvsQCD + 1e-8) + pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HbbvsQCD / (1 - pfParticleNetFromMiniAODAK8DiscriminatorsJetTags_HbbvsQCD + 1e-8) + 1e-8)",
        }
        cfg.y_score = exist_variable_dict[(sname, bname)]
    elif routine == "PNet":
        # for non-MD ParticleNet
        exist_variable_dict = {
            "QCD": " + ".join(
                [f"pfParticleNetJetTags_probQCD{n}" for n in ["b", "bb", "c", "cc", "others"]]
            ),
            **{
                f"TTBAR{n}": f"pfParticleNetJetTags_probT{n}"
                for n in ["bc", "bcq", "bel", "bmu", "bq", "bqq", "bta"]
            },
            "TTBARhad": " + ".join(
                [f"pfParticleNetJetTags_probT{n}" for n in ["bcq", "bqq", "bc", "bq"]]
            ),
            "TTBARel": " + ".join([f"pfParticleNetJetTags_probT{n}" for n in ["bel"]]),
            "TTBARmu": " + ".join([f"pfParticleNetJetTags_probT{n}" for n in ["bmu"]]),
            "TTBARta": " + ".join([f"pfParticleNetJetTags_probT{n}" for n in ["bta"]]),
            "TTBARbWall": " + ".join(
                [
                    f"pfParticleNetJetTags_prob{n}"
                    for n in ["Tbc", "Tbcq", "Tbel", "Tbmu", "Tbq", "Tbqq", "Tbta"]
                ]
            ),
            "TTBARbWhad": " + ".join(
                [f"pfParticleNetJetTags_probT{n}" for n in ["bcq", "bqq", "bc", "bq"]]
            ),
            "TTBARbWQQ": " + ".join([f"pfParticleNetJetTags_probT{n}" for n in ["bcq", "bqq"]]),
            "TTBARbWev": " + ".join([f"pfParticleNetJetTags_probT{n}" for n in ["bel"]]),
            "TTBARbWmv": " + ".join([f"pfParticleNetJetTags_probT{n}" for n in ["bmu"]]),
            "TTBARbWta": " + ".join([f"pfParticleNetJetTags_probT{n}" for n in ["bta"]]),
            "TTBARbWtauhv": " + ".join(
                [f"pfParticleNetJetTags_probT{n}" for n in ["bta"]]
            ),  ## special
            "ZQQ": " + ".join([f"pfParticleNetJetTags_probZ{n}" for n in ["bb", "cc", "qq"]]),
            "WQQ": " + ".join([f"pfParticleNetJetTags_probW{n}" for n in ["cq", "qq"]]),
        }
        exist_variable_dict.update(
            {
                k.replace("TTBAR", "SMTTBARSL"): v
                for k, v in exist_variable_dict.items()
                if k.startswith("TTBAR")
            }
        )
        cfg.y_score = f"({exist_variable_dict[sname]}) / (({exist_variable_dict[sname]}) + ({exist_variable_dict[bname]}))"
    elif routine == "GloParT1":
        exist_variable_dict = {
            "QCD": " + ".join(
                [
                    f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probQCD{n}"
                    for n in ["b", "bb", "c", "cc", "others"]
                ]
            ),
            **{
                f"HIGGS2P{n}": f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}"
                for n in ["bb", "cc", "ss", "qq", "tauhtaue", "tauhtaum", "tauhtauh"]
            },
            **{
                f"HIG{n}": f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}"
                for n in ["bb", "cc", "ss", "qq", "tauhtaue", "tauhtaum", "tauhtauh"]
            },
            "HWWQQQQ": "pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}0c + pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}1c + pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}2c".format(
                n="WqqWqq"
            ),
            "HWWQQQ": "pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}0c + pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}1c + pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}2c".format(
                n="WqqWq"
            ),
            "HWWQQev": "pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}0c + pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}1c".format(
                n="WqqWev"
            ),
            "HWWQQmv": "pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}0c + pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}1c".format(
                n="WqqWmv"
            ),
            "HWWQQtauev": "pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}0c + pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}1c".format(
                n="WqqWtauev"
            ),
            "HWWQQtaumv": "pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}0c + pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}1c".format(
                n="WqqWtaumv"
            ),
            "HWWQQtauhv": "pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}0c + pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}1c".format(
                n="WqqWtauhv"
            ),
            "HWWQQta": " + ".join(
                [
                    f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probH{n}"
                    for n in [
                        "WqqWtauev0c",
                        "WqqWtauev1c",
                        "WqqWtaumv0c",
                        "WqqWtaumv1c",
                        "WqqWtauhv0c",
                        "WqqWtauhv1c",
                    ]
                ]
            ),
            "TTBARbWall": " + ".join(
                [
                    f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probTop{n}"
                    for n in [
                        "bWqq0c",
                        "bWqq1c",
                        "bWq0c",
                        "bWq1c",
                        "bWev",
                        "bWmv",
                        "bWtauev",
                        "bWtaumv",
                        "bWtauhv",
                    ]
                ]
            ),
            "TTBARbWhad": " + ".join(
                [
                    f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probTop{n}"
                    for n in ["bWqq0c", "bWqq1c", "bWq0c", "bWq1c"]
                ]
            ),
            "TTBARbWQQ": " + ".join(
                [
                    f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probTop{n}"
                    for n in ["bWqq0c", "bWqq1c"]
                ]
            ),
            "TTBARbWev": " + ".join(
                [f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probTop{n}" for n in ["bWev"]]
            ),
            "TTBARbWmv": " + ".join(
                [f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probTop{n}" for n in ["bWmv"]]
            ),
            "TTBARbWta": " + ".join(
                [
                    f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probTop{n}"
                    for n in ["bWtauev", "bWtaumv", "bWtauhv"]
                ]
            ),
            "TTBARbWtauhv": " + ".join(
                [
                    f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probTop{n}"
                    for n in ["bWtauhv"]
                ]
            ),
            "TTBARall": " + ".join(
                [
                    f"pfMassDecorrelatedInclParticleTransformerV1JetTags_probTop{n}"
                    for n in [
                        "bWqq0c",
                        "bWqq1c",
                        "bWq0c",
                        "bWq1c",
                        "bWev",
                        "bWmv",
                        "bWtauev",
                        "bWtaumv",
                        "bWtauhv",
                    ]
                ]
            ),
        }
        exist_variable_dict.update(
            {
                k.replace("TTBAR", "SMTTBARSL"): v
                for k, v in exist_variable_dict.items()
                if k.startswith("TTBAR")
            }
        )
        cfg.y_score = f"({exist_variable_dict[sname]}) / (({exist_variable_dict[sname]}) + ({exist_variable_dict[bname]}))"
    elif routine == "GloParT2":
        exist_variable_dict = {
            "QCD": " + ".join(
                [
                    f"pfMassDecorrelatedInclParticleTransformerV2HidLayerJetTags_probQCD{n}"
                    for n in ["b", "bb", "c", "cc", "others"]
                ]
            ),
            **{
                f"HIGGS2P{n}": f"pfMassDecorrelatedInclParticleTransformerV2HidLayerJetTags_probH{n}"
                for n in [
                    "bb",
                    "cc",
                    "qq",
                    "ss",
                    "bc",
                    "cs",
                    "bs",
                    "gg",
                    "ee",
                    "mm",
                    "tauhtaue",
                    "tauhtaum",
                    "tauhtauh",
                ]
            },
            **{
                f"HIG{n}": f"pfMassDecorrelatedInclParticleTransformerV2HidLayerJetTags_probH{n}"
                for n in [
                    "bb",
                    "cc",
                    "qq",
                    "ss",
                    "bc",
                    "cs",
                    "bs",
                    "gg",
                    "ee",
                    "mm",
                    "tauhtaue",
                    "tauhtaum",
                    "tauhtauh",
                ]
            },
        }
        cfg.y_score = f"({exist_variable_dict[sname]}) / (({exist_variable_dict[sname]}) + ({exist_variable_dict[bname]}))"
    elif routine.endswith("_nonmd"):
        if sname.startswith("TTBAR"):
            # existing top class defination is fine
            pass
        if sname == "ZQQ":
            cfg.y_score_sig = (
                "score_label_Z_bb + score_label_Z_cc + score_label_Z_ss + score_label_Z_qq"
            )
            cfg.y_score = f"({cfg.y_score_sig}) / (({cfg.y_score_sig}) + ({cfg.y_score_bkg}))"
        elif sname == "WQQ":
            cfg.y_score_sig = "score_label_W_cs + score_label_W_qq"
            cfg.y_score = f"({cfg.y_score_sig}) / (({cfg.y_score_sig}) + ({cfg.y_score_bkg}))"
        else:
            raise RuntimeError("wrong opt for _nonmd routine")

    # special treatment for XX and X*X* variables
    if routine_config == "WW":
        cfg.label += r" ($H\to W W^*$ discr.)"
    elif routine_config == "WxWx":
        cfg.y_score = re.sub(r"H_WW", "H_WxWx", cfg.y_score)
        cfg.label += r" ($H\to W_x W_x$ discr.)"
    elif routine_config == "WxWxStar":
        cfg.y_score = re.sub(r"H_WW", "H_WxWxStar", cfg.y_score)
        cfg.label += r" ($H\to W_x^* W_x^{(*)}$ discr.)"
    elif routine_config == "WWallmodes":
        y_score_sig_2 = re.sub(r"H_WW", "H_WxWx", cfg.y_score_sig)
        y_score_sig_3 = re.sub(r"H_WW", "H_WxWxStar", cfg.y_score_sig)
        cfg.y_score = f"({cfg.y_score_sig} + {y_score_sig_2} + {y_score_sig_3}) / (({cfg.y_score_sig} + {y_score_sig_2} + {y_score_sig_3}) + ({cfg.y_score_bkg}))"
        cfg.label += r" ($H\to WW$ all-mode discr.)"

    # special weight parameters for ZQQ and WQQ tagging
    if routine in ["v2", "v3"]:
        if sname == "ZQQ":
            cfg.y_score_sig = "score_label_H_bb + score_label_H_cc + score_label_H_ss + 2 * score_label_H_qq"  # H_qq score * 2
            cfg.y_score = f"({cfg.y_score_sig}) / (({cfg.y_score_sig}) + ({cfg.y_score_bkg}))"
        elif sname == "Zll":
            cfg.y_score_sig = "score_label_H_ss + 2 * score_label_H_qq"  # H_qq score * 2
            cfg.y_score = f"({cfg.y_score_sig}) / (({cfg.y_score_sig}) + ({cfg.y_score_bkg}))"
        elif sname == "WQQ":  # or sname == 'SMTTBARSLQQ':
            cfg.y_score_sig = "score_label_H_cs + (score_label_H_ss + 2*score_label_H_qq) * (1/3)"  # use scores constructed from stored Nano branches
            # cfg.y_score_sig = 'score_label_H_cs + score_label_H_qq'
            cfg.y_score = f"({cfg.y_score_sig}) / (({cfg.y_score_sig}) + ({cfg.y_score_bkg}))"
        elif sname == "TTBARbWhad":
            cfg.y_score_sig = "score_label_Top_bWcs + score_label_Top_bWqq + 1 * (score_label_Top_bWc + score_label_Top_bWs + 2 * score_label_Top_bWq)"
            cfg.y_score = f"({cfg.y_score_sig}) / (({cfg.y_score_sig}) + ({cfg.y_score_bkg}))"

    # special beta3+ class splitting
    if routine.startswith("v3beta3") or routine.startswith("v3beta4") or routine == "v3":
        cfg.y_score = re.sub(
            r"score_label_Top_bW([a-zA-Z0-9_]+)",
            r"(score_label_Top_bWp\1 + score_label_Top_bWm\1)",
            cfg.y_score,
        )
        cfg.y_score = re.sub(
            r"score_label_Top_W([a-zA-Z0-9_]+)",
            r"(score_label_Top_Wp\1 + score_label_Top_Wm\1)",
            cfg.y_score,
        )
        cfg.y_score = re.sub(
            r"score_label_H_bc", r"(score_label_Hp_bc + score_label_Hm_bc)", cfg.y_score
        )
        cfg.y_score = re.sub(
            r"score_label_H_cs", r"(score_label_Hp_cs + score_label_Hm_cs)", cfg.y_score
        )

    # for testing W jets from SM ttbar sample: truth W jets have Top_W labels
    if sname == "SMTTBARSLQQ":
        cfg.y_sig_flag = "(fj_label == 10) | (fj_label == 11)"

    # for testing gg final states:
    if sname == "HIGgg":
        if routine_config is None:
            cfg.label += " (Hgg vs QCD)"
        elif routine_config == "ggonly":
            cfg.y_score = f"(score_label_H_gg)"
            cfg.label += " (Hgg)"
        elif routine_config == "1mqcdothers":
            cfg.y_score = f"(1 - score_label_QCD_others)"
            cfg.label += " (1 - QCDothers)"
        elif routine_config == "ggvsQCDf":
            cfg.y_score = f"(score_label_H_gg) / ((score_label_H_gg) + (score_label_QCD_bb + score_label_QCD_cc + score_label_QCD_b + score_label_QCD_c))"
            cfg.label += " (Hgg vs QCD:bb+cc+b+c)"
        elif routine_config == "1":
            cfg.y_sig1 = "score_label_H_gg + 0.2*score_label_H_WW_qqqq"
            cfg.y_bkg1 = "score_label_QCD_bb + score_label_QCD_cc + score_label_QCD_b + score_label_QCD_c + score_label_QCD_others"
            cfg.y_score = f"({cfg.y_sig1}) / (({cfg.y_sig1}) + ({cfg.y_bkg1}))"
            cfg.label += " (Hgg+4q vs QCD)"
    return cfg

In [ ]:
def plot_roc_curve(
    sname,
    bname,
    samples,
    pt="low",
    mass="higgs",
    xmin=0,
    ymin=1e-4,
    xtext=0.5,
    store_plot=False,
    subdir="",
    genm=None,
    year="2017",
    plot_kwargs_list=None,
):
    colorlist = ["blue", "red", "green", "darkorange", "darkviolet", "cyan"]
    mpl.rcParams["axes.prop_cycle"] = cycler(color=colorlist)
    f, ax = plt.subplots()
    year_legend_map = {
        "2017": "2017 (UL)",
        "2017PUPPIv18": "2017 (UL PUPPIv18)",
        "2023": "2023",
        "2023BPix": "2023BPix",
        "2023FixLTS": "2023FixLTS",
    }
    cms_label = None  # "Preliminary"
    hep.cms.label(
        cms_label,
        data=False,
        loc=1,
        rlabel="{year} ({com} TeV)".format(
            year=year_legend_map[year],
            com="13.6" if any(y in year for y in ["2022", "2023"]) else "13",
        ),
        ax=ax,
        fontname="sans-serif",
    )
    from scipy.interpolate import interp1d

    # create dataframe
    for isam, sam in enumerate(samples):
        opt = f"{sam}:{sname}:{bname}"
        config = getConfig(opt, pt=pt, mass=mass, year=year)

        print(opt)
        print(
            config.filelist["sig"][0].rsplit("/", 1)[0],
            [l.split("/")[-1] for l in config.filelist["sig"]],
            [l.split("/")[-1] for l in config.filelist["bkg"]],
            config.y_sig_flag,
            config.y_bkg_flag,
            config.y_score,
            config.base_cut,
            config.mass_cut,
        )

        dfs = uproot.lazy(config.filelist["sig"])
        dfb = uproot.lazy(config.filelist["bkg"])
        # apply final cut to the dataset
        dfs = dfs[
            ak.numexpr.evaluate(
                f"({config.y_sig_flag}) & ({config.base_cut}) & ({config.mass_cut})", dfs
            )
        ]
        dfb = dfb[
            ak.numexpr.evaluate(
                f"({config.y_bkg_flag}) & ({config.base_cut}) & ({config.mass_cut})", dfb
            )
        ]

        y_score_s = ak.numexpr.evaluate(config.y_score, dfs)
        y_score_b = ak.numexpr.evaluate(config.y_score, dfb)
        y_score = ak.concatenate([y_score_s, y_score_b])

        if genm is not None:
            genm_cut_str = f"abs((fj_gen_mass / {genm}) - 1) < 1e-3"
            y_score_s = y_score_s[ak.numexpr.evaluate(genm_cut_str, dfs)]
            if bname.startswith("HIG") or bname.startswith("HWW"):
                y_score_b = y_score_b[ak.numexpr.evaluate(genm_cut_str, dfb)]
            y_score = ak.concatenate([y_score_s, y_score_b])

        # compute quantile on bkg
        qs = np.array([1e-1, 1e-2, 1e-3, 1e-4])
        thres = np.quantile(y_score_b, 1 - qs)
        print("quantile thresholds", thres)
        print("n_left_signal", [ak.sum(y_score_s > t) for t in thres])

        # ## make plots
        # f0, ax0 = plt.subplots(figsize=(10, 10))
        # nbin, xmin, xmax = 1000, 0., 1.
        # hist = bh.Histogram(bh.axis.Regular(nbin, xmin, xmax), storage=bh.storage.Weight())
        # hist.fill(y_score_s)
        # content, yerr = hist.view().value, np.sqrt(hist.view().variance)
        # hep.histplot(content / sum(content), bins=hist.axes[0].edges, yerr=yerr / sum(content), label=sname)
        # hist.fill(y_score_b)
        # content, yerr = hist.view().value, np.sqrt(hist.view().variance)
        # hep.histplot(content / sum(content), bins=hist.axes[0].edges, yerr=yerr / sum(content), label=bname)
        # ax0.set_xlabel(f'{sam} {sname} vs {bname} discr.', ha='right', x=1.0); ax0.set_ylabel('A.U.', ha='right', y=1.0)
        # ax0.legend()
        # ax0.set_xlim(0, 1)
        # ax0.set_ylim(1e-6, 1e1), ax0.set_yscale('log')
        # ###

        y_true = ak.concatenate(
            [ak.ones_like(y_score_s, dtype=bool), ak.zeros_like(y_score_b, dtype=bool)]
        )
        print(ak.sum(y_true), ak.sum(~y_true))

        fpr, tpr, _thres = m.roc_curve(y_true, y_score)
        auc = m.auc(fpr, tpr)
        ax.plot(
            tpr,
            fpr,
            label=config.label,  # + r" $\bf{(AUC: %.4f)}$" % auc,
            **(plot_kwargs_list[isam] if plot_kwargs_list is not None else {}),
        )
        # draw 1% and 0.1% lines
        func = interp1d(fpr, tpr)
        ax.plot([0, 1], [0.01, 0.01], ":", color="grey")
        ax.plot([func(0.01), func(0.01)], [0, 0.01], ":", color="grey")
        ax.plot([0, 1], [0.001, 0.001], "-.", color="darkgrey")
        ax.plot([func(0.001), func(0.001)], [0, 0.001], "-.", color="darkgrey")

    ax.legend(
        loc="upper right", labelspacing=0.8, prop={"size": 19}
    )  # , bbox_to_anchor=(0.03, 0.6)) #prop={'size': 22},
    ax.set_yscale("log")
    ax.set_xlim(xmin, 1)
    ax.set_ylim(ymin, 1)
    ax.set_xlabel("Signal efficiency", ha="right", x=1.0)
    ax.set_ylabel("Background efficiency", ha="right", y=1.0)
    x_text, y_text = 0.0, 0.65
    # x_text, y_text = xtext, 0.05
    if genm is not None:
        ax.text(
            x_text + 0.03,
            y_text + 0.18,
            config.title,
            fontsize=22,
            fontweight="bold",
            transform=ax.transAxes,
        )
        ax.text(
            x_text + 0.03,
            y_text + 0.12,
            r"$m_{H} = %d$ GeV" % genm,
            fontsize=18,
            fontweight="bold",
            transform=ax.transAxes,
        )
        ax.text(x_text + 0.03, y_text + 0.06, config.subtitle, fontsize=18, transform=ax.transAxes)
        ax.text(x_text + 0.03, y_text + 0.00, config.subtitle2, fontsize=18, transform=ax.transAxes)
    else:
        ax.text(
            x_text + 0.03,
            y_text + 0.13,
            config.title,
            fontsize=22,
            fontweight="bold",
            transform=ax.transAxes,
        )
        ax.text(x_text + 0.03, y_text + 0.06, config.subtitle, fontsize=18, transform=ax.transAxes)
        ax.text(x_text + 0.03, y_text + 0.00, config.subtitle2, fontsize=18, transform=ax.transAxes)
    if store_plot:
        os.makedirs(f"./plots/{subdir}", exist_ok=True)
        plt.savefig(
            f'./plots/{subdir}/roc_{sname}_{bname}_{"-".join(samples).replace(";","")}_{pt}_{mass}_{year}_{genm}.pdf',
            bbox_inches="tight",
        )
        plt.savefig(
            f'./plots/{subdir}/roc_{sname}_{bname}_{"-".join(samples).replace(";","")}_{pt}_{mass}_{year}_{genm}.png',
            bbox_inches="tight",
        )
        print()

In [ ]:
year = "2023"
for pt, ptsig, mass, genm in [("low", "LO", "higgs", 125)]:
    # # for pt, ptsig, mass, genm in [('high', 'HI', 'higgs', 125)]:
    # plot_roc_curve(f'HIG{ptsig}bb', f'QCD', samples=['PNetMD', 'v2', 'v3'], pt=pt, mass=mass, year=year, store_plot=True, subdir='v3beta4_hqq', xtext=0.5, genm=genm)
    plot_roc_curve(
        f"HIG{ptsig}bb",
        f"QCD",
        samples=["PNetMD", "v3"],
        pt=pt,
        mass=mass,
        year=year,
        store_plot=True,
        subdir="v3beta4_hqq",
        xtext=0.5,
        genm=genm,
        plot_kwargs_list=[{"ls": "--"}, {"ls": "-"}],
    )

# Mass regression (Hbb)

In [ ]:
label_dict = {
    "QCD": "QCD",
    "XQQbb": r"H$\rightarrow$bb",
    "XQQcc": r"H$\rightarrow$cc",
    "XQQss": r"H$\rightarrow$ss",
    "XQQqq": r"H$\rightarrow$qq (u/d/s)",
    "XQQtauhtaue": r"H$\rightarrow\tau_h\tau_e$",
    "XQQtauhtaum": r"H$\rightarrow\tau_h\tau_\mu$",
    "XQQtauhtauh": r"H$\rightarrow\tau_h\tau_h$",
}
samples = {  # (label, path)
    "sd": (
        "Soft-drop",
        "/ceph/cms/store/user/woodson/glopart-plotter/predict/ak8_MD_inclv10beta4_ul_manual.nlayer10.vispart_as_resid.ddp4-bs640-lr1p2e-3.nepoch100.farm221.best_epoch",
    ),
    "pnetofc": (
        "ParticleNet",
        "/ceph/cms/store/user/woodson/glopart-plotter/predict/ak8_MD_inclv10beta4_ul_manual.nlayer10.vispart_as_resid.ddp4-bs640-lr1p2e-3.nepoch100.farm221.best_epoch",
    ),
    # 'v3.modevisible': ('GloParT 3 (generic)', '/ceph/cms/store/user/woodson/glopart-plotter/predict/ak8_MD_inclv10beta4_ul_manual.nlayer10.vispart_as_resid.ddp4-bs640-lr1p2e-3.nepoch100.farm221.best_epoch'),
    "v3.modex2p": (
        "GloParT",
        "/ceph/cms/store/user/woodson/glopart-plotter/predict/ak8_MD_inclv10beta4_ul_manual.nlayer10.vispart_as_resid.ddp4-bs640-lr1p2e-3.nepoch100.farm221.best_epoch",
    ),
}

# discr_sigeff = 30
# for discr_sigeff in [30, 100]:
#     for year in ['2017', '2017PUPPIv18', '2023', '2023BPix']:
#         for genm in [50, 125, 250, 350]:
for discr_sigeff in [30]:
    for year in ["2023"]:
        for genm in [125]:
            gen_mode = "vispart"

            year_file_id = {
                "2017": "",
                "2023": "run3_2023_",
                "2023BPix": "run3_2023bpix_",
                "2017PUPPIv18": "run2_repuppi_",
            }[year]
            year_legend_map = {
                "2017": "2017 (UL)",
                "2017PUPPIv18": "2017 (UL PUPPIv18)",
                "2023": "2023",
                "2023BPix": "2023BPix",
            }

            # for computing the discr value
            dff = uproot.lazy(
                glob.glob(
                    f"/ceph/cms/store/user/woodson/glopart-plotter/predict/ak8_MD_inclv10beta4_ul_manual.nlayer10.vispart_as_resid.ddp4-bs640-lr1p2e-3.nepoch100.farm221.best_epoch/pred_{year_file_id}higlo*.root"
                )
            )
            dff = dff[dff.fj_gen_mass == genm]
            dff = dff[ak.numexpr.evaluate("(fj_pt>200) & (fj_pt<2500)", dff)]
            pnet_discr = (
                lambda s: f"(pfMassDecorrelatedParticleNetJetTags_probX{s}) / ((pfMassDecorrelatedParticleNetJetTags_probX{s}) + (pfMassDecorrelatedParticleNetJetTags_probQCDb + pfMassDecorrelatedParticleNetJetTags_probQCDbb + pfMassDecorrelatedParticleNetJetTags_probQCDc + pfMassDecorrelatedParticleNetJetTags_probQCDcc + pfMassDecorrelatedParticleNetJetTags_probQCDothers))"
            )
            glopart3_discr = (
                lambda s: f"(score_label_H_{s}) / ((score_label_H_{s}) + (score_label_QCD_b + score_label_QCD_bb + score_label_QCD_c + score_label_QCD_cc + score_label_QCD_others))"
            )

            truth_selection = {
                "H_bb": "(fj_label == 17)",
                "H_cc": "(fj_label == 18)",
                "H_qq": "(fj_label == 19) | (fj_label == 20)",
                "H_tauhtaue": "(fj_label == 27)",
                "H_tauhtaum": "(fj_label == 28)",
                "H_tauhtauh": "(fj_label == 29)",
            }
            categories = {
                "H_bb": ["H_bb"],
                "H_cc": ["H_cc"],
                "H_qq": ["H_qq"],
                "H_tauhtaue": ["H_tauhtaue"],
            }
            categories_2p = {
                "H_bb": ["H_bb", "H_cc", "H_ss", "H_qq", "Hp_cs", "Hm_cs"],
                "H_cc": ["H_bb", "H_cc", "H_ss", "H_qq", "Hp_cs", "Hm_cs"],
                "H_qq": ["H_bb", "H_cc", "H_ss", "H_qq", "Hp_cs", "Hm_cs"],
            }

            ## define GloParT 3 reg computing utilities
            weighted = lambda values, weights: np.sum(values * weights, axis=0) / np.sum(
                weights, axis=0
            )
            get_values = lambda df_, categories_: np.array(
                [
                    ak.numexpr.evaluate(f"output_target_res_mass_factor_label_{l}", df_)
                    for l in categories_
                ]
            )
            # get_weights = lambda df_, categories_: np.array([ak.numexpr.evaluate(f'score_label_{l}', df_) for l in categories_])
            get_weights = lambda df_, categories_: np.array(
                [
                    ak.numexpr.evaluate(f"score_label_{l}", df_) * (2.0 if l == "H_qq" else 1.0)
                    for l in categories_
                ]
            )

            if genm == 50:
                nbin, xmin, xmax = 100, 0, 100
            elif genm == 125:
                nbin, xmin, xmax = 50, 0, 250
            elif genm == 250 or genm == 350:
                nbin, xmin, xmax = 200, 0, 500

            colorlist = [
                "black",
                "blue",
                "red",
                "green",
                "darkorange",
                "darkviolet",
                "cyan",
                "greenyellow",
            ]
            mpl.rcParams["axes.prop_cycle"] = cycler(color=colorlist)

            # for ch in ['H_bb', 'H_cc', 'H_qq']:
            for ch in ["H_bb"]:
                # first compute the discr value
                dff_truth = dff[ak.numexpr.evaluate(truth_selection[ch], dff)]
                val = ak.numexpr.evaluate(pnet_discr(ch[2:]), dff_truth)  # H_bb, H_cc, H_qq
                discr_thres = np.quantile(val, 1 - discr_sigeff / 100)
                print(ch, f"{discr_sigeff=}, {discr_thres=}")

                # no SD cut!
                preselection = {
                    "H_bb": f"(abs(fj_eta)<2.4) & (fj_pt>200) & (fj_pt<2500) & ({pnet_discr('bb')} > {discr_thres})",
                    "H_cc": f"(abs(fj_eta)<2.4) & (fj_pt>200) & (fj_pt<2500) & ({pnet_discr('cc')} > {discr_thres})",
                    "H_qq": f"(abs(fj_eta)<2.4) & (fj_pt>200) & (fj_pt<2500) & ({pnet_discr('qq')} > {discr_thres})",
                    # 'H_tauhtaue': f"(fj_pt>200) & (fj_pt<2500) & ({pnet_discr('tauhtaue')} > {discr_thres})",
                }

                f, ax = plt.subplots(figsize=(10, 10))
                cms_label = None  # "Preliminary"
                hep.cms.label(
                    cms_label,
                    data=False,
                    loc=1,
                    rlabel="{year} ({com} TeV)".format(
                        year=year_legend_map[year],
                        com="13.6" if any(y in year for y in ["2022", "2023"]) else "13",
                    ),
                    ax=ax,
                    fontname="sans-serif",
                )

                for n, (lab, path) in samples.items():
                    df = uproot.lazy(glob.glob(path + f"/pred_{year_file_id}higlo*.root"))
                    df = df[df.fj_gen_mass == genm]
                    df = df[ak.numexpr.evaluate(preselection[ch], df)]

                    _df = df[ak.numexpr.evaluate(truth_selection[ch], df)]
                    print(f"{len(_df)=}")

                    if n == "sd":
                        var = "fj_sdmass"
                        array = ak.numexpr.evaluate(var, _df)
                        ls = "-."
                    elif n == "pnetofc":
                        var = "pfParticleNetMassRegressionJetTags_mass"
                        array = ak.numexpr.evaluate(var, _df)
                        ls = "--"
                    elif n == "v1":
                        var = "pfMassDecorrelatedInclParticleTransformerV1JetTags_mass"
                        array = ak.numexpr.evaluate(var, _df)
                    elif n == "v2":
                        var = (
                            "output_target_res_mass_factor * fj_mass"
                            if gen_mode == "res"
                            else "output_target_parts_mass_factor * fj_mass"
                        )
                        array = ak.numexpr.evaluate(var, _df)
                    elif n.startswith("v3"):  # multiple modes!!
                        if n == "v3.modexbb":
                            categories_map = categories
                        elif n == "v3.modex2p":
                            categories_map = categories_2p
                            ls = "-"
                        if n in ["v3.modexbb", "v3.modex2p"]:
                            # factor = weighted(get_values(_df, categories_mode[n.split('.')[-1]]), get_weights(_df, categories_mode[n.split('.')[-1]]))
                            factor = weighted(
                                get_values(_df, categories_map[ch]),
                                get_weights(_df, categories_map[ch]),
                            )
                            array = factor * _df.fj_mass
                            # vispart_as_resid
                            array = array + ak.numexpr.evaluate(
                                "output_target_parts_mass_factor * fj_mass", _df
                            )
                            var = "weighted: %s, vispart_as_resid" % str(categories_map[ch])
                        elif n == "v3.modevisible":
                            var = "output_target_parts_mass_factor * fj_mass"
                            array = ak.numexpr.evaluate(var, _df)

                    print(ch, n, var)

                    hist = bh.Histogram(
                        bh.axis.Regular(nbin, xmin, xmax), storage=bh.storage.Weight()
                    )
                    hist.fill(array)
                    # content, yerr = hist.view().value / sum(hist.view().value), np.sqrt(hist.view().variance) / sum(hist.view().value)
                    content, yerr = hist.view().value, np.sqrt(hist.view().variance)

                    # calculate guassian KDE metric
                    from scipy.stats import gaussian_kde

                    array = ak.to_numpy(array)
                    kde = gaussian_kde(array, bw_method=0.2)
                    x_grid = np.linspace(xmin, xmax, 10001)
                    y = kde.evaluate(x_grid)
                    halfmax = y.max() / 2
                    maxpos = y.argmax()
                    leftpos = (np.abs(y[:maxpos] - halfmax)).argmin()
                    rightpos = (np.abs(y[maxpos:] - halfmax)).argmin() + maxpos
                    peak_value = x_grid[maxpos]
                    fwhm = x_grid[rightpos] - x_grid[leftpos]
                    # print(f'Peak value: {peak_value}', f'FWHM: {fwhm}')
                    if n == "pnetofc" and genm in [50, 250, 350]:
                        fwhm = "N/A"

                    # hep.histplot(content, bins=hist.axes[0].edges, yerr=yerr, label=lab + r'  $\bf{(\mu=%.1f,\,\sigma=%.1f,\,\epsilon_{%d\pm\! 10}=%.1f\%% )}$' % (np.mean(array), np.std(array), genm, sum((array > genm-10) & (array < genm+10)) / len(array) * 100))
                    hep.histplot(
                        content / np.sum(content),
                        ls=ls,
                        bins=hist.axes[0].edges,
                        yerr=yerr / np.sum(content),
                        label=lab
                        + r" $\bf{(m_{peak}=%.1f,\,FWHM=%s)}$"
                        % (peak_value, f"{fwhm:.1f}" if not isinstance(fwhm, str) else fwhm),
                    )

                ax.legend(
                    loc="upper right",
                    labelspacing=0.4,
                    prop={"size": 18},
                    bbox_to_anchor=(1.0, 0.9),
                )
                ax.set_xlabel(r"$m_{reco}$ [GeV]", ha="right", x=1.0)
                ax.set_ylabel("A.U.", ha="right", y=1.0)

                ax.set_xlim(xmin, xmax)
                ax.set_ylim(0, ax.get_ylim()[1] * 1.5)
                ytext = 0.4
                ax.text(
                    0.03,
                    0.27 + ytext,
                    label_dict[f"XQQ{ch[2:]}"],
                    fontsize=22,
                    fontweight="bold",
                    transform=ax.transAxes,
                )
                ax.text(
                    0.03,
                    0.22 + ytext,
                    r"$m_{H} = %d$ GeV" % genm,
                    fontsize=18,
                    fontweight="bold",
                    transform=ax.transAxes,
                )
                ax.text(
                    0.03,
                    0.16 + ytext,
                    r"$\epsilon_{sig} = %d$%s selected" % (discr_sigeff, "%"),
                    fontsize=18,
                    transform=ax.transAxes,
                )
                ax.text(0.03, 0.11 + ytext, r"by ParticleNet", fontsize=18, transform=ax.transAxes)
                ax.text(
                    0.03,
                    0.04 + ytext,
                    r"$p_T > 200$ GeV, $|\eta|$ < 2.4",
                    fontsize=18,
                    transform=ax.transAxes,
                )
                # ax.text(0.03, 0.04+ytext, r'$p_T > 200$ GeV, $m_{SD}$ > 0, $\sigma_{m_{reco}} > 20$ GeV', fontsize=18, transform=ax.transAxes)
                os.makedirs("./plots/v3beta4_reg", exist_ok=True)
                plt.savefig(
                    f"./plots/v3beta4_reg/regmass_hqq_{ch}_m{genm}_sigeff{discr_sigeff}_{year}.pdf",
                    bbox_inches="tight",
                )
                plt.savefig(
                    f"./plots/v3beta4_reg/regmass_hqq_{ch}_m{genm}_sigeff{discr_sigeff}_{year}.png",
                    bbox_inches="tight",
                )